In [14]:
# STEP 1: Imports & Data Loading
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook
from bokeh.models import ColumnDataSource
output_notebook()

# Load the dataset
df = pd.read_csv("dataset.csv")

# STEP 2: Preprocessing
df.rename(columns={"SystemCodeNumber": "Lot_ID"}, inplace=True)
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")

# Map categorical variables to numerical weights
traffic_level_weights = {"low": 1, "average": 2, "high": 3}
vehicle_type_weights = {"car": 1, "bike": 0.8, "truck": 1.2, "cycle": 0.5}

df["Traffic_Level"] = df["TrafficConditionNearby"].map(traffic_level_weights)
df["Vehicle_Weight"] = df["VehicleType"].map(vehicle_type_weights)

# Drop rows with any NA values from mapped columns
df = df.dropna(subset=["Traffic_Level", "Vehicle_Weight", "Occupancy", "Capacity", "QueueLength"])

# STEP 3: Compute Raw Demand Score
def compute_demand(row, alpha=0.6, beta=0.3, gamma=0.5, delta=0.2, epsilon=0.1):
    occupancy_rate = float(row["Occupancy"]) / float(row["Capacity"])
    queue = row["QueueLength"]
    traffic = row["Traffic_Level"]
    special_day = row["IsSpecialDay"]
    vehicle_weight = row["Vehicle_Weight"]
    
    demand = (
        alpha * occupancy_rate +
        beta * queue +
        gamma * traffic +
        delta * special_day +
        epsilon * vehicle_weight
    )
    return demand

df["Raw_Demand"] = df.apply(compute_demand, axis=1)

# STEP 4: Normalize Demand Score & Price
df["Demand_Score"] = (df["Raw_Demand"] - df["Raw_Demand"].min()) / (df["Raw_Demand"].max() - df["Raw_Demand"].min())
base_price = 20  # base price in $
max_increase = 15  # maximum increase over base price
df["Price"] = base_price + df["Demand_Score"] * max_increase

# STEP 5: Visualization using Bokeh
from bokeh.layouts import layout

# Use the first valid Lot_ID to visualize
sample_lot = df["Lot_ID"].iloc[0]
lot_df = df[df["Lot_ID"] == sample_lot].sort_values("Timestamp")

source = ColumnDataSource(lot_df)

p = figure(title=f"Dynamic Pricing Over Time: Lot {sample_lot}",
           x_axis_type="datetime", width=800, height=400)
p.line(x="Timestamp", y="Price", source=source, line_width=2, color="darkblue", legend_label="Price ($)")
p.circle(x="Timestamp", y="Price", source=source, size=5, color="orange")

p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price ($)"
p.legend.location = "top_left"

show(p)


Loading BokehJS ...

⚠️ No data available after preprocessing. Please check CSV and column types.
